<a href="https://colab.research.google.com/github/talpt/pyton/blob/main/Maestro_LRC_300_Price_CrossOver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/rongardF/tvdatafeed matplotlib openpyxl ta
!pip install tradingview-screener==2.5.0

import pandas as pd
import numpy as np
from tvDatafeed import TvDatafeed, Interval
from tradingview_screener import get_all_symbols
import warnings
import matplotlib.pyplot as plt
from tabulate import tabulate

warnings.simplefilter(action='ignore', category=FutureWarning)

# TvDatafeed nesnesi oluşturma
tv = TvDatafeed()

# Türkiye piyasasındaki tüm sembolleri almak için
try:
    Hisseler = get_all_symbols(market='turkey')
    Hisseler = [symbol.replace('BIST:', '') for symbol in Hisseler]
    Hisseler = sorted(Hisseler)
except Exception as e:
    print(f"Hata alındı: {e}")

# Lineer regresyon fonksiyonu
def linreg(series, length):
    x = np.arange(length)
    def linear_regression(y):
        A = np.vstack([x, np.ones(len(x))]).T
        m, c = np.linalg.lstsq(A, y, rcond=None)[0]
        return m * x[-1] + c
    return series.rolling(length).apply(linear_regression, raw=False)

# Kullanıcının zaman dilimini seçmesini sağlayan fonksiyon
def get_interval_choice():
    intervals = {
        '1': ('15 Dakika', Interval.in_15_minute),
        '2': ('30 Dakika', Interval.in_30_minute),
        '3': ('45 Dakika', Interval.in_45_minute),
        '4': ('1 Saat', Interval.in_1_hour),
        '5': ('2 Saat', Interval.in_2_hour),
        '6': ('4 Saat', Interval.in_4_hour),
        '7': ('Günlük', Interval.in_daily),
        '8': ('Haftalık', Interval.in_weekly),
        '9': ('Aylık', Interval.in_monthly)
    }

    print("Lütfen bir zaman dilimi seçin:")
    for key, value in intervals.items():
        print(f"{key}: {value[0]}")

    choice = input("Seçiminizi yapın (1-9): ")

    if choice in intervals:
        print(f"{intervals[choice][0]} seçildi.")
        return intervals[choice][1], intervals[choice][0]  # Hem interval hem de adı döndür
    else:
        print("Geçersiz seçim. Varsayılan olarak günlük (1D) seçildi.")
        return Interval.in_daily, 'Günlük'

# Üst mü yoksa alt kanal mı taranacağını seçme
def get_scan_type():
    print("Tarama tipi seçin:")
    print("1: Alt kanal kesişimi")
    print("2: Üst kanal kesişimi")
    print("3: Orta bant kesişimi")  # Yeni seçenek

    scan_choice = input("Seçiminizi yapın (1-3): ")

    if scan_choice == '1':
        return 'lower'
    elif scan_choice == '2':
        return 'upper'
    elif scan_choice == '3':
        return 'middle'
    else:
        print("Geçersiz seçim. Varsayılan olarak alt kanal kesişimi seçildi.")
        return 'lower'

# Ayarlar
n_bars = 1000  # Geçmişteki bar sayısı
length = 300   # Lineer regresyon için periyot

# Zaman dilimini kullanıcıdan almak
interval, interval_name = get_interval_choice()

# Tarama tipini kullanıcıdan almak
scan_type = get_scan_type()

results = []

# Her sembol için döngü
for symbol in Hisseler:
    try:
        # TradingView'dan veri çekme
        data = tv.get_hist(symbol=symbol, exchange='BIST', interval=interval, n_bars=n_bars)
        if data is None or data.empty:
            continue

        # Linear Regression ve Deviation hesaplama
        data['linreg_high'] = linreg(data['high'], length)
        data['linreg_low'] = linreg(data['low'], length)
        data['dev_low'] = -data['low'].rolling(window=length).std() + data['linreg_low']
        data['dev_high'] = data['high'].rolling(window=length).std() + data['linreg_high']

        # Orta bant hesaplama
        data['middle_band'] = (data['linreg_high'] + data['linreg_low']) / 2

        # Fiyatın alt kanal kesişimini kontrol etme
        data['cross_up'] = (data['close'].shift(1) < data['dev_low'].shift(1)) & (data['close'] > data['dev_low'])

        # Fiyatın üst kanal kesişimini kontrol etme
        data['cross_down'] = (data['close'].shift(1) > data['dev_high'].shift(1)) & (data['close'] < data['dev_high'])

        # Fiyatın dev_high üzerinde olup olmadığını kontrol etme
        data['above_high'] = data['close'] > data['dev_high']

        # Fiyatın orta bandı kesişimini kontrol etme
        data['middle_band_cross'] = (data['close'].shift(1) < data['middle_band'].shift(1)) & (data['close'] > data['middle_band'])

        # Tarama tipine göre filtreleme
        if scan_type == 'lower' and data.iloc[-1]['cross_up']:
            result_message = f"{symbol} alt kanal kesişimi sağlandı. Son Fiyat: {data.iloc[-1]['close']}, Kesişim Zamanı: {data.index[-1]}, Periyot: {interval_name}"
            print(result_message)  # Anlık sonuç
            results.append({
                'Hisse Adı': symbol,
                'Son Fiyat': data.iloc[-1]['close'],
                'Kanal Tipi': 'Alt Kanal',
                'Kesişim Zamanı': data.index[-1],
                'Periyot': interval_name    # Periyot bilgisi eklendi
            })
        elif scan_type == 'upper' and (data.iloc[-1]['cross_down'] or data.iloc[-1]['above_high']):
            result_message = f"{symbol} üst kanal kesişimi sağlandı. Son Fiyat: {data.iloc[-1]['close']}, Kesişim Zamanı: {data.index[-1]}, Periyot: {interval_name}"
            print(result_message)  # Anlık sonuç
            results.append({
                'Hisse Adı': symbol,
                'Son Fiyat': data.iloc[-1]['close'],
                'Kanal Tipi': 'Üst Kanal',
                'Kesişim Zamanı': data.index[-1],
                'Periyot': interval_name    # Periyot bilgisi eklendi
            })
        elif scan_type == 'middle' and data.iloc[-1]['middle_band_cross']:
            result_message = f"{symbol} orta bant kesişimi sağlandı. Son Fiyat: {data.iloc[-1]['close']}, Kesişim Zamanı: {data.index[-1]}, Periyot: {interval_name}"
            print(result_message)  # Anlık sonuç
            results.append({
                'Hisse Adı': symbol,
                'Son Fiyat': data.iloc[-1]['close'],
                'Kanal Tipi': 'Orta Bant',
                'Kesişim Zamanı': data.index[-1],
                'Periyot': interval_name    # Periyot bilgisi eklendi
            })

    except Exception as e:
        print(f"Hata alındı {symbol}: {e}")

# Sonuçları tablo olarak gösterme
if results:
    df_results = pd.DataFrame(results)
    print("\nKanal kesişimi yapan hisseler:")
    print(tabulate(df_results, headers='keys', tablefmt='grid'))

    # Görsel olarak tabloyu gösterme
    fig, ax = plt.subplots(figsize=(12, 8))
    ax.axis('tight')
    ax.axis('off')
    ax.table(cellText=df_results.values, colLabels=df_results.columns, cellLoc='center', loc='center')
    plt.show()
else:
    print("\nHiçbir hisse belirtilen kanal kesişim koşulunu sağlamıyor.")




  Cloning https://github.com/rongardF/tvdatafeed to /tmp/pip-req-build-0x6sbrz1
  Running command git clone --filter=blob:none --quiet https://github.com/rongardF/tvdatafeed /tmp/pip-req-build-0x6sbrz1
  Resolved https://github.com/rongardF/tvdatafeed to commit e6f6aaa7de439ac6e454d9b26d2760ded8dc4923
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for tvdatafeed: filename=tvdatafeed-2.1.0-py3-none-any.whl size=17533 sha256=7fb86540084d5c17305dbe446b398ac25757d136136f9ed5c164e8ec1dd297a2
  Stored in directory: /tmp/pip-ephem-wheel-cache-nbujpkr5/wheels/0a/ba/99/b27476fd1e4caf0dd70445cdc6798195d3b90005a1501a12f7
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=ea755c639f77e33426b061d3d8e50f15af3d6a550cb33cbe3e37cfe7b03decef
  Stored in directory: /root/.cache/pip/wheels/a1/d7/29/7781cc5eb9a3659d032d7d15bdd0f49d07d2b24fec29f44bc4
Successfully built tvdatafeed ta
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5

Lütfen bir zaman dilimi seçin:
1: 15 Dakika
2: 30 Dakika
3: 45 Dakika
4: 1 Saat
5: 2 Saat
6: 4 Saat
7: Günlük
8: Haftalık
9: Aylık
Seçiminizi yapın (1-9): 9
Aylık seçildi.
Tarama tipi seçin:
1: Alt kanal kesişimi
2: Üst kanal kesişimi
3: Orta bant kesişimi
Seçiminizi yapın (1-3): 3
HEKTS orta bant kesişimi sağlandı. Son Fiyat: 3.44, Kesişim Zamanı: 2025-03-03 06:00:00, Periyot: Aylık


KeyboardInterrupt: 